## Create CONSTANTS

In [70]:
DATASET = [
    {
        'Tid': 'T1',
        'Item': ['a', 'b', 'c', 'd'],
        'Quantity': [5,2,1,2]
    },
    {
        'Tid': 'T2',
        'Item': ['a', 'c', 'd', 'g'],
        'Quantity': [1,1,1,3]
    },
    {
        'Tid': 'T3',
        'Item': ['a', 'c', 'f'],
        'Quantity': [1,1,1]
    },
    {
        'Tid': 'T4',
        'Item': ['a', 'f', 'g'],
        'Quantity': [1,4,2]
    },
    {
        'Tid': 'T5',
        'Item': ['a', 'g'],
        'Quantity': [1,2]
    },
    {
        'Tid': 'T6',
        'Item': ['b', 'c', 'd', 'e'],
        'Quantity': [3,2,3,1]
    },
    {
        'Tid': 'T7',
        'Item': ['c', 'e'],
        'Quantity': [6,4]
    },
    {
        'Tid': 'T8',
        'Item': ['e', 'f'],
        'Quantity': [1,3]
    }
]
UNIT_UTILITY = {
    'a': 3,
    'b': 6,
    'c': -3,
    'd': 12,
    'e': -5,
    'f': -2,
    'g': -1
}

EPM = {
    'min_util': 25,
    'min_per': 1,
    'max_per': 6,
    'min_avg': 1,
    'max_avg': 5
}

### Show global variable

In [71]:
for i in DATASET:
    print(i)

{'Tid': 'T1', 'Item': ['a', 'b', 'c', 'd'], 'Quantity': [5, 2, 1, 2]}
{'Tid': 'T2', 'Item': ['a', 'c', 'd', 'g'], 'Quantity': [1, 1, 1, 3]}
{'Tid': 'T3', 'Item': ['a', 'c', 'f'], 'Quantity': [1, 1, 1]}
{'Tid': 'T4', 'Item': ['a', 'f', 'g'], 'Quantity': [1, 4, 2]}
{'Tid': 'T5', 'Item': ['a', 'g'], 'Quantity': [1, 2]}
{'Tid': 'T6', 'Item': ['b', 'c', 'd', 'e'], 'Quantity': [3, 2, 3, 1]}
{'Tid': 'T7', 'Item': ['c', 'e'], 'Quantity': [6, 4]}
{'Tid': 'T8', 'Item': ['e', 'f'], 'Quantity': [1, 3]}


In [72]:
print(UNIT_UTILITY)

{'a': 3, 'b': 6, 'c': -3, 'd': 12, 'e': -5, 'f': -2, 'g': -1}


In [73]:
print(EPM)

{'min_util': 25, 'min_per': 1, 'max_per': 6, 'min_avg': 1, 'max_avg': 5}


## Preparation Procedure

The Transaction Weight Utility (TWU) of item \( i \) is given by:

Def 6
Def 4
\[
{TWU}(i) = \sum_{{X \in T_j \\ T_j \subseteq D}} P_U(T_j)
\]

calculate maxPer(i), avgPer(i)
Def 1, Def 2

rearrange order
Def 7


In [74]:
def create_periodic_high_util_itemset(dataset: list, unit_utility: dict, epm: dict) -> list:
    # for i in dataset:
    #     print(i)
    # calculate TWU of each item in dataset
    twu_dict = dict()
    twu_transaction_dict = dict()
    ps_dict = dict()
    transaction_dict = dict()
    for i in dataset:
        twu_transaction_dict[i['Tid']] = 0
        for j in range(len(i['Item'])):
            if i['Item'][j] not in twu_dict:
                twu_dict[i['Item'][j]] = 0
                transaction_dict[i['Item'][j]] = []
                ps_dict[i['Item'][j]] = []

            twu_transaction_dict[i['Tid']] += i['Quantity'][j] * unit_utility[i['Item'][j]] if unit_utility[i['Item'][j]] > 0 else 0
            transaction_dict[i['Item'][j]].append(int(i['Tid'][1:]))

        for j in range(len(i['Item'])):
            twu_dict[i['Item'][j]] += twu_transaction_dict[i['Tid']]

    # print(twu_transaction_dict)
    for i in transaction_dict.keys():
        ps_dict[i].append(transaction_dict[i][0])
        for j in range(1, len(transaction_dict[i])):
            ps_dict[i].append(transaction_dict[i][j] - transaction_dict[i][j - 1])

    temp_dataset = list()
    eucs_dict = dict()
    order_set = set()

    for i in dataset:
        temp_item_quantity = list()
        for j in range(len(i['Item'])):
            avg_item = sum(ps_dict[i['Item'][j]])/len(ps_dict[i['Item'][j]])
            # print((i['Item'][j], i['Quantity'][j], unit_utility[i['Item'][j]], twu_dict[i['Item'][j]]))
            if twu_dict[i['Item'][j]] > epm['min_util'] and max(ps_dict[i['Item'][j]]) <= epm['max_per']\
            and avg_item >= epm['min_avg']:

                temp_item_quantity.append((i['Item'][j], i['Quantity'][j], unit_utility[i['Item'][j]], twu_dict[i['Item'][j]]))
                order_set.add((i['Item'][j], unit_utility[i['Item'][j]], twu_dict[i['Item'][j]]))

        temp_item_quantity.sort(key=lambda x: x[3])
        print(temp_item_quantity)
        temp_item_quantity.sort(key=lambda x: x[2])
        print(temp_item_quantity)
        # print(i)
        temp_dataset.append(
            {
                'Tid': i['Tid'],
                'Item': list(map(lambda x: x[0], temp_item_quantity)),
                'Quantity': list(map(lambda x: x[1], temp_item_quantity))
            }
        )
        # print(i, end='\n\n')
    order_list = list(order_set)
    order_list.sort(key=lambda x:x[2])
    order_list.sort(key=lambda x:x[1])
    order_list = [i[0] for i in order_list]

    for i in temp_dataset:
        for j in range(len(i['Item']) - 1):
          for k in range(j + 1, len(i['Item'])):
            if i['Item'][j] + i['Item'][k] not in eucs_dict:
                eucs_dict[i['Item'][j] + i['Item'][k]] = 0
            eucs_dict[i['Item'][j] + i['Item'][k]] += twu_transaction_dict[i['Tid']]

    return temp_dataset, eucs_dict, order_list

In [75]:
NEW_DATASET, EUCS_LIST, ORDER_LIST = create_periodic_high_util_itemset(dataset=DATASET, unit_utility=UNIT_UTILITY, epm=EPM)

[('a', 5, 3, 75), ('b', 2, 6, 105), ('d', 2, 12, 120), ('c', 1, -3, 123)]
[('c', 1, -3, 123), ('a', 5, 3, 75), ('b', 2, 6, 105), ('d', 2, 12, 120)]
[('a', 1, 3, 75), ('d', 1, 12, 120), ('c', 1, -3, 123)]
[('c', 1, -3, 123), ('a', 1, 3, 75), ('d', 1, 12, 120)]
[('a', 1, 3, 75), ('c', 1, -3, 123)]
[('c', 1, -3, 123), ('a', 1, 3, 75)]
[('a', 1, 3, 75)]
[('a', 1, 3, 75)]
[('a', 1, 3, 75)]
[('a', 1, 3, 75)]
[('e', 1, -5, 54), ('b', 3, 6, 105), ('d', 3, 12, 120), ('c', 2, -3, 123)]
[('e', 1, -5, 54), ('c', 2, -3, 123), ('b', 3, 6, 105), ('d', 3, 12, 120)]
[('e', 4, -5, 54), ('c', 6, -3, 123)]
[('e', 4, -5, 54), ('c', 6, -3, 123)]
[('e', 1, -5, 54)]
[('e', 1, -5, 54)]


In [76]:
for i in NEW_DATASET:
    print(i)

{'Tid': 'T1', 'Item': ['c', 'a', 'b', 'd'], 'Quantity': [1, 5, 2, 2]}
{'Tid': 'T2', 'Item': ['c', 'a', 'd'], 'Quantity': [1, 1, 1]}
{'Tid': 'T3', 'Item': ['c', 'a'], 'Quantity': [1, 1]}
{'Tid': 'T4', 'Item': ['a'], 'Quantity': [1]}
{'Tid': 'T5', 'Item': ['a'], 'Quantity': [1]}
{'Tid': 'T6', 'Item': ['e', 'c', 'b', 'd'], 'Quantity': [1, 2, 3, 3]}
{'Tid': 'T7', 'Item': ['e', 'c'], 'Quantity': [4, 6]}
{'Tid': 'T8', 'Item': ['e'], 'Quantity': [1]}


In [77]:
print(EUCS_LIST)

{'ca': 69, 'cb': 105, 'cd': 120, 'ab': 51, 'ad': 66, 'bd': 105, 'ec': 54, 'eb': 54, 'ed': 54}


In [78]:
print(ORDER_LIST)

['e', 'c', 'a', 'b', 'd']


## Searching stage

In [79]:
def construct(prefix_list: list, x: list, y: list) -> list:
    pre_pointer = 0
    x_pointer = 0
    y_pointer = 0
    result = list()

    while x_pointer < len(x) and y_pointer < len(y):
        if x[x_pointer][0] < y[y_pointer][0]:
            x_pointer += 1
        elif x[x_pointer][0] > y[y_pointer][0]:
            y_pointer += 1
        else:
            if prefix_list != None:
                while prefix_list[pre_pointer][0] < x[x_pointer][0]:
                    pre_pointer += 1
                result.append([pre_pointer, x[x_pointer][1] + y[y_pointer][1] - prefix_list[pre_pointer][1], min(x[x_pointer][2], y[y_pointer][2])])
                x_pointer += 1
                y_pointer += 1
                pre_pointer += 1
            else:
                result.append([x_pointer, x[x_pointer][1] + y[y_pointer][1], min(x[x_pointer][2], y[y_pointer][2])])
                x_pointer += 1
                y_pointer += 1
    return result

def utility_condition(x, minutil):
    return sum(i[1] for i in x) >= minutil

def calculate_periodic(x):
    periodic = [x[0][0] + 1]
    keys = x
    for i in range(len(keys) - 1):
        periodic.append(keys[i+1][0] - keys[i][0])
    return max(periodic), min(periodic), sum(periodic) / len(periodic)

def periodic_condition(x, minper, maxper, minavg, maxavg):
    max_i, min_i, avg_i = calculate_periodic(x)
    return min_i >= minper and max_i <= maxper and avg_i >= minavg and avg_i <= maxavg

def ru_condition(x, minutil):
    return sum(i[2] for i in x) >= minutil

def max_per_condition(x, maxper):
    max_x, min_x, avg_x = calculate_periodic(x)
    return max_x <= maxper

def avg_per_condition(x, maxavg):
    max_x, min_x, avg_x = calculate_periodic(x)
    return avg_x <= maxavg

def get_name(X, Y):
    # return X + Y if X + Y in eucs_list else Y + X
    return X + Y

def eucp_condition(X, Y, eucs_list, minutil):
    items_name = get_name(X, Y)
    return eucs_list[items_name] >= minutil

# def is_mlist(Y):
#     return True if isinstance(Y[0], str) and isinstance(Y[1], str) else False

# def du_compute(Y, X):
#     if is_mlist(Y):
#         return sum(x[2][-1] + x[2][1] for x in Y) + sum(x[1] for x in X)
#     else:
#         return sum(x[-1] + x[1] for x in Y) + sum(x[1] for x in X)
    
# def gen_mlist_1(X, Y_MLIST, DU):
#     mlist = Y_MLIST
#     for i in mlist
    
#     return None

# def gen_mlist_2(X, Y_LIST, P, DU):
#     return None


In [80]:
def phm_seach(prefix : str, prefix_list: list, lists: dict, order_list: list) -> set:
    result = set()
    key_lists = list()

    for i in order_list:
      if i in lists:
        key_lists.append(i)

    # print(prefix, key_lists, order_list)
    for i in key_lists:
        # print(i)
        # print(i, lists[i])
        # print(lists[i], end='\t')
        # print(i, utility_condition(lists[i], EPM['min_util']), periodic_condition(lists[i], EPM['min_per'], EPM['max_per'], EPM['min_avg'], EPM['max_avg'])\
        #       , calculate_periodic(lists[i]))
        if utility_condition(lists[i], EPM['min_util']) and periodic_condition(lists[i], EPM['min_per'], EPM['max_per'], EPM['min_avg'], EPM['max_avg']):
            result.add(i)
        # print(lists[i])
        # print(EPM)
        if ru_condition(lists[i], EPM['min_util']) and max_per_condition(lists[i], EPM['max_per']) and avg_per_condition(lists[i], EPM['max_avg']):
            new_list = dict()
            new_order_list = list()
            for j in key_lists[key_lists.index(i) + 1:]:
                if get_name(i[-1], j[-1]) not in EUCS_LIST:
                  continue
                if eucp_condition(i[-1],j[-1], EUCS_LIST, EPM['min_util']):

                    z = get_name(i[-1], j[-1]) if prefix == None else prefix + get_name(i[-1], j[-1])

                    z_list = construct(prefix_list, lists[i], lists[j])

                    new_list[z] = z_list
                    new_order_list.append(z)
            result.update(phm_seach(i, lists[i], new_list, new_order_list))

    return result

In [81]:
print(UNIT_UTILITY)

for i in NEW_DATASET:
    print(i)

{'a': 3, 'b': 6, 'c': -3, 'd': 12, 'e': -5, 'f': -2, 'g': -1}
{'Tid': 'T1', 'Item': ['c', 'a', 'b', 'd'], 'Quantity': [1, 5, 2, 2]}
{'Tid': 'T2', 'Item': ['c', 'a', 'd'], 'Quantity': [1, 1, 1]}
{'Tid': 'T3', 'Item': ['c', 'a'], 'Quantity': [1, 1]}
{'Tid': 'T4', 'Item': ['a'], 'Quantity': [1]}
{'Tid': 'T5', 'Item': ['a'], 'Quantity': [1]}
{'Tid': 'T6', 'Item': ['e', 'c', 'b', 'd'], 'Quantity': [1, 2, 3, 3]}
{'Tid': 'T7', 'Item': ['e', 'c'], 'Quantity': [4, 6]}
{'Tid': 'T8', 'Item': ['e'], 'Quantity': [1]}


In [82]:
utility_list = dict()
pnu_utility_list = dict()
for i in NEW_DATASET:
    total = 0
    for j in range(len(i['Item']) - 1, -1, -1):

        if i['Item'][j] not in utility_list:
            utility_list[i['Item'][j]] = list()
            pnu_utility_list[i['Item'][j]] = list()

        utility_list[i['Item'][j]].append([int(i['Tid'][1:]), UNIT_UTILITY[i['Item'][j]]*i['Quantity'][j], total])
        
        if UNIT_UTILITY[i['Item'][j]] < 0:
            pnu_utility_list[i['Item'][j]].append([int(i['Tid'][1:]), 0, UNIT_UTILITY[i['Item'][j]]*i['Quantity'][j], total])
        else:
            pnu_utility_list[i['Item'][j]].append([int(i['Tid'][1:]), UNIT_UTILITY[i['Item'][j]]*i['Quantity'][j], 0, total])

        total += UNIT_UTILITY[i['Item'][j]]*i['Quantity'][j] if UNIT_UTILITY[i['Item'][j]] > 0 else 0
for i in utility_list.keys():
    print(i, utility_list[i])

d [[1, 24, 0], [2, 12, 0], [6, 36, 0]]
b [[1, 12, 24], [6, 18, 36]]
a [[1, 15, 36], [2, 3, 12], [3, 3, 0], [4, 3, 0], [5, 3, 0]]
c [[1, -3, 51], [2, -3, 15], [3, -3, 3], [6, -6, 54], [7, -18, 0]]
e [[6, -5, 54], [7, -20, 0], [8, -5, 0]]


In [83]:
for i, j in pnu_utility_list.items():
    print(i, j)

d [[1, 24, 0, 0], [2, 12, 0, 0], [6, 36, 0, 0]]
b [[1, 12, 0, 24], [6, 18, 0, 36]]
a [[1, 15, 0, 36], [2, 3, 0, 12], [3, 3, 0, 0], [4, 3, 0, 0], [5, 3, 0, 0]]
c [[1, 0, -3, 51], [2, 0, -3, 15], [3, 0, -3, 3], [6, 0, -6, 54], [7, 0, -18, 0]]
e [[6, 0, -5, 54], [7, 0, -20, 0], [8, 0, -5, 0]]


In [84]:
result = phm_seach(None, None, utility_list, ORDER_LIST)

In [85]:
print(result)

{'cd', 'ad', 'ab', 'cbd', 'b', 'bd', 'a', 'd', 'cad'}


In [86]:
def phm_plus_search(prefix : str, prefix_list: list, lists: dict, item_utility: list, order_list: list) -> set:
    result = set()
    key_lists = list()

    for i in order_list:
      if i in lists:
        key_lists.append(i)

    for i in key_lists:
        temp_list_of_i = list(map(lambda x: [x[0], x[1] + x[2], x[3]], lists[i]))
        if utility_condition(temp_list_of_i, EPM['min_util']) and periodic_condition(temp_list_of_i, EPM['min_per'], EPM['max_per'], EPM['min_avg'], EPM['max_avg']):
            result.add(i)
        # print(lists[i])
        # print(EPM)
        if ru_condition(temp_list_of_i, EPM['min_util']) and max_per_condition(temp_list_of_i, EPM['max_per']) and avg_per_condition(temp_list_of_i, EPM['max_avg']):
            new_list = dict()
            new_order_list = list()
            for j in key_lists[key_lists.index(i) + 1:]:
                if get_name(i[-1], j[-1]) not in EUCS_LIST:
                  continue
                if eucp_condition(i[-1],j[-1], EUCS_LIST, EPM['min_util']):
                    
                    du_i_j = du_compute(lists[j], lists[i])
                    z = get_name(i[-1], j[-1]) if prefix == None else prefix + get_name(i[-1], j[-1])

                    if du_i_j >= EPM['min_util']:
                      if is_mlist(lists[j]):
                        z_list = construct(lists[j][-1], lists[i], lists[j][1])
                      else:
                        z_list = construct(prefix_list, lists[i], lists[j])
                    else:
                      if is_mlist(lists[j]):
                        z_list = gen_mlist_1(i, lists[j], du_i_j)
                      else:
                        z_list = gen_mlist_2(i, lists[j], prefix, du_i_j)

                    new_list[z] = z_list
                    new_order_list.append(z)
            result.update(phm_plus_search(i, lists[i], new_list, item_utility, new_order_list))

    return result